# 03 - Checking Data Consistency

The purpose of this document is to ensure there is **no Noisy Data** in the interim dataset as the output of the *ETL.py* script.

## Setting Up Project Directory

In [1]:
from jupyter_init import setup

setup()

from src_code.config import *

## Loading Dataset

In [10]:
import pandas as pd
import numpy as np

from src_code.versioning import VersionedFileManager

# DF_PATH = ETL_MAPPINGS['train']['current_newest']
version_manager = VersionedFileManager(
    src_dir=INTERIM_DATA_DIR, 
    file_name="train_extended.feather")
# DF_PATH 

# ---- LOAD ----
df = pd.read_feather(version_manager.current_newest)
print(f"Loaded dataframe with {len(df)} rows and {len(df.columns)} columns\n")

df.dtypes

Loaded dataframe with 500 rows and 132 columns



datetime                 datetime64[us, pytz.FixedOffset(-120)]
commit                                                   object
repo                                                     object
filepath                                                 object
content                                                  object
                                          ...                  
tfidf_values property                                   float64
tfidf_widepanel                                         float64
tfidf_work                                              float64
tfidf_working                                           float64
tfidf_working docs                                      float64
Length: 132, dtype: object

## Converting NumpyArray -> List



In [3]:
#Convert the NumPy arrays back to Python lists
for col in ['code_embed', 'msg_embed', 'methods', 'lines']:
    # Use .apply(list) or .apply(lambda x: x.tolist()) for robustness
    df[col] = df[col].apply(list)

# print(df['content'].head(5))

## Missing Value Audit

In [4]:
print("## 1. Missing Values per Column")
nulls = df.isnull().sum().sort_values(ascending=False)
print(nulls.to_markdown())

## 1. Missing Values per Column
|                             |   0 |
|:----------------------------|----:|
| datetime                    |   0 |
| commit                      |   0 |
| repo                        |   0 |
| filepath                    |   0 |
| content                     |   0 |
| methods                     |   0 |
| lines                       |   0 |
| author_email                |   0 |
| canonical_datetime          |   0 |
| author_exp_pre              |   0 |
| author_recent_activity_pre  |   0 |
| label                       |   0 |
| loc_added                   |   0 |
| loc_deleted                 |   0 |
| files_changed               |   0 |
| hunks_count                 |   0 |
| msg_len                     |   0 |
| has_fix_kw                  |   0 |
| has_bug_kw                  |   0 |
| ast_delta                   |   0 |
| complexity_delta            |   0 |
| max_func_change             |   0 |
| time_since_last_change      |   0 |
| todo            

## Primary Key Integrity

In [5]:
print("## 2. Primary Key Uniqueness Check")
key_cols = ["repo", "commit", "filepath"]

dupes = df.duplicated(subset=key_cols).sum()
print(f"Duplicate key rows: {dupes}")

## 2. Primary Key Uniqueness Check
Duplicate key rows: 0


## Label Distribution

In [6]:
print("## 3. Label Distribution")
print(df['label'].value_counts(normalize=True).to_markdown())

## 3. Label Distribution
|   label |   proportion |
|--------:|-------------:|
|       0 |         0.92 |
|       1 |         0.08 |


## Repository Distribution (Imbalance Check)

In [7]:
print("## 4. Repository Distribution")
repo_dist = df['repo'].value_counts(normalize=True)
print(repo_dist.to_markdown())

## 4. Repository Distribution
| repo    |   proportion |
|:--------|-------------:|
| ansible |   0.306752   |
| sentry  |   0.228907   |
| core    |   0.20873    |
| pandas  |   0.103982   |
| ray     |   0.0874358  |
| airflow |   0.0572519  |
| numpy   |   0.00694071 |


## Value Range Scan for Numeric Columns

Automatically detects:

- negatives where not allowed
- max values
- suspicious spikes

In [11]:
print("## 5. Numeric Column Range Scan")
num_cols = df.select_dtypes(include=[np.number]).columns

ranges = pd.DataFrame({
    "min": df[num_cols].min(),
    "median": df[num_cols].median(),
    "mean": df[num_cols].mean(),
    "max": df[num_cols].max()
})

print(ranges.to_markdown())

## 5. Numeric Column Range Scan
|                             |   min |   median |           mean |             max |
|:----------------------------|------:|---------:|---------------:|----------------:|
| author_exp_pre              |     0 |    244.5 |   244.534      |   494           |
| author_recent_activity_pre  |     0 |     39.5 |    44.078      |   111           |
| label                       |     0 |      0   |     0.08       |     1           |
| loc_added                   |     0 |     39   |   107.026      |  1378           |
| loc_deleted                 |     0 |     61.5 |   136.872      |  1893           |
| files_changed               |     0 |      4   |     5.238      |    19           |
| hunks_count                 |     0 |     24   |    41.932      |   862           |
| msg_len                     |    13 |     57   |    79.414      |   290           |
| has_fix_kw                  |     0 |      0   |     0.136      |     1           |
| has_bug_kw          

In [9]:
print(num_cols.tolist())

['author_exp_pre', 'author_recent_activity_pre', 'label', 'loc_added', 'loc_deleted', 'files_changed', 'hunks_count', 'msg_len', 'has_fix_kw', 'has_bug_kw', 'ast_delta', 'complexity_delta', 'max_func_change', 'time_since_last_change', 'todo', 'fixme', 'try', 'except', 'raise', 'recent_churn', 'tfidf_11de', 'tfidf_11de ac95', 'tfidf_7de3', 'tfidf_7de3 11de', 'tfidf_ac95', 'tfidf_ac95 d976489f1ece', 'tfidf_added', 'tfidf_address', 'tfidf_address gh', 'tfidf_bug', 'tfidf_bugfix', 'tfidf_changed', 'tfidf_changed series', 'tfidf_cleanup', 'tfidf_code', 'tfidf_code reorg', 'tfidf_columns', 'tfidf_com', 'tfidf_com svn', 'tfidf_consistent', 'tfidf_consistent datamatrix', 'tfidf_constructor', 'tfidf_core', 'tfidf_coverage', 'tfidf_cython', 'tfidf_cython code', 'tfidf_d5231056', 'tfidf_d5231056 7de3', 'tfidf_d976489f1ece', 'tfidf_dataframe', 'tfidf_datamatrix', 'tfidf_doc', 'tfidf_docs', 'tfidf_enh', 'tfidf_fancy', 'tfidf_fixed', 'tfidf_functions', 'tfidf_gh', 'tfidf_git', 'tfidf_git svn', 'tfid

## Check Columns Expected to Be Non-Negative

In [9]:
non_negative_cols = [
    "loc_added", "loc_deleted",
    "files_changed", "hunks_count",
    "msg_len", "ast_delta",
    "complexity_delta", "max_func_change",
    "author_exp_pre", "author_recent_activity_pre",
    "todo", "fixme", "try", "except", "raise",
    "recent_churn"
]

print("## 6. Negative Value Check")
for col in non_negative_cols:
    bad = (df[col] < 0).sum()
    print(f"{col}: {bad} negative values")

## 6. Negative Value Check
loc_added: 0 negative values
loc_deleted: 0 negative values
files_changed: 0 negative values
hunks_count: 0 negative values
msg_len: 0 negative values
ast_delta: 0 negative values
complexity_delta: 0 negative values
max_func_change: 0 negative values
author_exp_pre: 0 negative values
author_recent_activity_pre: 0 negative values
todo: 0 negative values
fixme: 0 negative values
try: 0 negative values
except: 0 negative values
raise: 0 negative values
recent_churn: 0 negative values


## Suspicious Feature Check: time_since_last_change

In [10]:
print("## 7. time_since_last_change Outliers")
tslc = df["time_since_last_change"]

print(f"Negative values: {(tslc < 0).sum()}")
print(f"99.9% quantile: {tslc.quantile(0.999)}")
print(f"Min: {tslc.min()}")
print(f"Max: {tslc.max()}")

## 7. time_since_last_change Outliers
Negative values: 66
99.9% quantile: 828201.0000000028
Min: -641148
Max: 35334961


### Understanding the Feature

*time_since_last_change = c.committed_date - last_time*

Where:
- c.committed_date = current commit timestamp (UNIX seconds)
- last_time = timestamp of first parent commit

So the feature = time difference between consecutive commits.

This represents how much time passed between commits in a repo.

### Why Negative Values?

Meaning:

- Some commits appear to be ~4.6 days negative (-396,818 sec)
- Some commits appear to be ~77 days ahead (6.6M sec)

This is expected when real Git data is used.

Git timestamps can go backwards because:

#### (1) Rebased or rewritten history

During rebases, old commits appear “later” than newer ones.

*WHY?*

When you merge/rebase, Git does not reorder commits chronologically.
Instead, it preserves the logical order of development.

#### (2) Merge parents

You use only the first parent:

if c.parents:
    last_time = c.parents[0].committed_date


But merges may introduce non-linear time ordering.

#### (3) Clock drift

Different authors → different local machine clocks.

#### (4) Shallow clones or incomplete history

If the repo is shallow-fetched, parent commits may have weird timestamps.

None of this indicates your extraction is wrong.



## Binary Flag Integity

In [11]:
print("## 8. Binary Columns Integrity")
bin_cols = ["has_fix_kw", "has_bug_kw"]

for col in bin_cols:
    bad = df[~df[col].isin([0,1])]
    print(f"{col}: {len(bad)} invalid values")

## 8. Binary Columns Integrity
has_fix_kw: 0 invalid values
has_bug_kw: 0 invalid values


## Embedding Consistency Check

Ensure:

- no None
- all lists
- identical dimensionality

In [12]:
# # Convert the NumPy arrays back to Python lists
# for col in ['code_embed', 'msg_embed']:
#     # Use .apply(list) or .apply(lambda x: x.tolist()) for robustness
#     df[col] = df[col].apply(list)

print(type(df.loc[0, 'code_embed']))
print(type(df.loc[0, 'msg_embed']))

print("## 9. Embedding Structural Checks")

# None count
print("code_embed None count:", df['code_embed'].isna().sum())
print("msg_embed None count:", df['msg_embed'].isna().sum())

# Check if all are lists
print("\nNon-list code_embed rows:", (~df['code_embed'].apply(lambda x: isinstance(x, list))).sum())
print("Non-list msg_embed rows:", (~df['msg_embed'].apply(lambda x: isinstance(x, list))).sum())

# Check dimensionality
dims = df['code_embed'].apply(lambda x: len(x) if isinstance(x, list) else None)
print("\nEmbedding dimensionality distribution:")
print(dims.value_counts().head())

<class 'list'>
<class 'list'>
## 9. Embedding Structural Checks
code_embed None count: 0
msg_embed None count: 0

Non-list code_embed rows: 0
Non-list msg_embed rows: 0

Embedding dimensionality distribution:
code_embed
768    139611
Name: count, dtype: int64


## Datetime Consistency

Check for:

- NaT values
- ordering sanity (commit should not be older than file's previous record)

In [13]:
print("## 10. Datetime Columns Audit")

date_cols = ["datetime", "canonical_datetime"]

for col in date_cols:
    print(f"{col}: NaT count = {df[col].isna().sum()}")
    print(f"{col}: min = {df[col].min()}, max = {df[col].max()}")

## 10. Datetime Columns Audit
datetime: NaT count = 0
datetime: min = 2005-09-20 01:00:26-02:00, max = 2022-01-03 09:41:30-02:00
canonical_datetime: NaT count = 0
canonical_datetime: min = 2005-09-20 03:00:26+00:00, max = 2022-01-04 01:08:58+00:00


In [14]:
print("## 12. Columns With Only One Unique Value")

for col in df.columns:
    # Convert arrays to tuples for uniqueness check
    if df[col].apply(lambda x: isinstance(x, (np.ndarray, list))).any():
        # Convert each element to a tuple (or a string representation)
        # print(col)
        unique_values = df[col].apply(lambda x: tuple(x) if isinstance(x, (np.ndarray, list)) else x)
    else:
        unique_values = df[col]

    if unique_values.nunique(dropna=True) == 1:
        print(f"⚠️ {col} has only one unique value")

## 12. Columns With Only One Unique Value


## Check Text Columns for Weirdness

Empty strings? Too short? Too long?

In [15]:
print("## 13. Text Field Checks")

if 'content' in df.columns:
    print("Empty content rows:", (df['content'].str.len() == 0).sum())
    print(df['content'].str.len().describe())

# Check 'methods' if it is a list column
if 'methods' in df.columns:
    # Use len() on the Python lists
    print("\nEmpty methods rows:", (df['methods'].apply(len) == 0).sum())
    print(df['methods'].apply(len).describe())

## 13. Text Field Checks
Empty content rows: 2528
count    139611.000000
mean       1847.522487
std        4625.184828
min           0.000000
25%         416.000000
50%         789.000000
75%        1771.000000
max      227258.000000
Name: content, dtype: float64

Empty methods rows: 40716
count    139611.000000
mean          2.004785
std           3.884402
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max         284.000000
Name: methods, dtype: float64


## Check For Impossible Values

In [16]:
print("## 14. Logical Consistency Checks")

# msg_len should match commit message length
if "msg_len" in df.columns:
    print("msg_len outliers (msg_len <= 0):", (df['msg_len'] <= 0).sum())

## 14. Logical Consistency Checks
msg_len outliers (msg_len <= 0): 0


## Filepath Sanity
Check for Windows vs POSIX weirdness.

In [17]:
# Normalize all filepaths to use forward slashes
df['filepath'] = df['filepath'].str.replace('\\', '/', regex=False)

# Check again for paths without a slash
bad_paths = df[~df['filepath'].str.contains('/')]
print("Filepaths without / (unexpected after normalization):", len(bad_paths))
print(df.loc[~df['filepath'].str.contains('/'), 'filepath'].head(20))

Filepaths without / (unexpected after normalization): 1297
11        setup.py
80        setup.py
81     setupegg.py
122       setup.py
123       setup.py
138       setup.py
144       setup.py
168       setup.py
278       setup.py
461       setup.py
462       setup.py
479       setup.py
497       setup.py
513       setup.py
514       setup.py
523       setup.py
524       setup.py
531       setup.py
562       setup.py
570       setup.py
Name: filepath, dtype: object


## Check Recent Churn for Extreme Outliers

In [18]:
print("## 16. recent_churn Outlier Scan")
print(df['recent_churn'].describe())
print("99.9% quantile:", df['recent_churn'].quantile(0.999))

## 16. recent_churn Outlier Scan
count    1.396110e+05
mean     1.611727e+04
std      8.762949e+04
min      0.000000e+00
25%      1.030000e+02
50%      1.292000e+03
75%      6.885000e+03
max      1.977985e+06
Name: recent_churn, dtype: float64
99.9% quantile: 1366319.1900000102


## Check Distribution of Code Activity Keywords

(todo, fixme, try/except/raise)

In [19]:
print("## 17. Keyword Column Distributions")
kw_cols = ["todo", "fixme", "try", "except", "raise"]

print(df[kw_cols].describe().T.to_markdown())

## 17. Keyword Column Distributions
|        |   count |       mean |        std |   min |   25% |   50% |   75% |    max |
|:-------|--------:|-----------:|-----------:|------:|------:|------:|------:|-------:|
| todo   |  139611 |  0.161986  |    2.43365 |     0 |     0 |     0 |     0 |    108 |
| fixme  |  139611 |  0.0431628 |    1.01885 |     0 |     0 |     0 |     0 |     52 |
| try    |  139611 | 49.1044    | 3164.77    |     0 |     0 |     0 |     1 | 251603 |
| except |  139611 |  5.32309   |  183.854   |     0 |     0 |     0 |     0 |   8444 |
| raise  |  139611 |  1.56658   |   40.5607  |     0 |     0 |     0 |     0 |   1855 |


## Duplicate commit-message / code-embed lengths

Check uniformity:

In [20]:
code_duplicates = df['code_embed'].apply(len).value_counts().head()
msg_duplicates = df['msg_embed'].apply(len).value_counts().head()

print(code_duplicates)
print(msg_duplicates)

code_embed
768    139611
Name: count, dtype: int64
msg_embed
768    139611
Name: count, dtype: int64


So all your embeddings being length 768 means:

- The model you are using outputs a 768-dimensional vector
- The embedding extraction process worked for every commit
- No corrupted or empty embeddings
- No input missing (no None, no NaN, no empty list, etc.)

## File extension distribution
This reveals if:

- non-Python files contaminate AST/keyword metrics
- certain extensions dominate bugs

In [21]:
df['ext'] = df['filepath'].str.split('.').str[-1]
df['ext'].value_counts()

ext
py     139486
pyi       125
Name: count, dtype: int64

This does not mean the entire Pandas repo only contains .py files.
It means:

All diffs included in your dataset modify .py or .pyi files.

The Defectors dataset:

- extracts commits that add/remove lines regarded as “faulty”
- focuses on logical/code changes, not text, docs, or build files
- filters out non-source changes for consistency

So your dataset is a filtered view of the repo, not the repo itself.

## Diff size sanity

Large mismatches could indicate:

- truncated diffs
- non-standard diff formatting
- metadata lines (prefix +++, ---, etc.)

In [22]:
(df["content"].str.count("\n") - df["loc_added"] - df["loc_deleted"]).describe()


count    139611.000000
mean       -175.923351
std         549.203149
min      -30859.000000
25%        -199.000000
50%         -39.000000
75%           7.000000
max        5359.000000
dtype: float64

The discrepancy between content line counts and loc_added + loc_deleted is normal in real Git data.

Reasons:

1. Multi-line statements / code folding in diffs
2. Partial line changes counted in hunks
3. Files with removed lines only (content now shorter)
4. Large diffs in a single commit skew the stats

No indication of extraction errors — these are just properties of real commit diffs.